In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import math
import re


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

### Useful Function

In [2]:
# This function turns all letters to lower case and convert features containing 'sk_id_' to string format
def clean_header(df):
    id_columns = re.compile('sk_id_')
    df.columns = map(str.lower, df.columns)
    for i in df.columns:
        if id_columns.search(i):
            df[i] = df[i].astype(str)
    return df

### Data Import

In [3]:
# Connect to my AWS server and 'project5' database
params = {
    'host': '3.130.17.136',
    'user': 'ubuntu',
    'port': 5432
}
connection = pg.connect(**params, dbname='project5')
cursor = connection.cursor()

In [4]:
# Query 'application_train' data using Postgresql
application_train_df = pd.read_sql("""

SELECT *
FROM application_train

""", connection)

In [5]:
# Use clean_header function to 'application_train_df'
clean_header(application_train_df);

In [6]:
# Create new dataframe that only contains defaulted loan
default_loans = application_train_df[application_train_df['target']==1]

In [7]:
# Create new dataframe that only contains paid-off loan
normal_loans = application_train_df[application_train_df['target']==0]

In [8]:
default_loans.head()

,sk_id_curr,target,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,name_type_suite,name_income_type,name_education_type,name_family_status,name_housing_type,region_population_relative,days_birth,days_employed,days_registration,days_id_publish,own_car_age,flag_mobil,flag_emp_phone,flag_work_phone,flag_cont_mobile,flag_phone,flag_email,occupation_type,cnt_fam_members,region_rating_client,region_rating_client_w_city,weekday_appr_process_start,hour_appr_process_start,reg_region_not_live_region,reg_region_not_work_region,live_region_not_work_region,reg_city_not_live_city,reg_city_not_work_city,live_city_not_work_city,organization_type,ext_source_1,ext_source_2,ext_source_3,apartments_avg,basementarea_avg,years_beginexpluatation_avg,years_build_avg,commonarea_avg,elevators_avg,entrances_avg,floorsmax_avg,floorsmin_avg,landarea_avg,livingapartments_avg,livingarea_avg,nonlivingapartments_avg,nonlivingarea_avg,apartments_mode,basementarea_mode,years_beginexpluatation_mode,years_build_mode,commonarea_mode,elevators_mode,entrances_mode,floorsmax_mode,floorsmin_mode,landarea_mode,livingapartments_mode,livingarea_mode,nonlivingapartments_mode,nonlivingarea_mode,apartments_medi,basementarea_medi,years_beginexpluatation_medi,years_build_medi,commonarea_medi,elevators_medi,entrances_medi,floorsmax_medi,floorsmin_medi,landarea_medi,livingapartments_medi,livingarea_medi,nonlivingapartments_medi,nonlivingarea_medi,fondkapremont_mode,housetype_mode,totalarea_mode,wallsmaterial_mode,emergencystate_mode,obs_30_cnt_social_circle,def_30_cnt_social_circle,obs_60_cnt_social_circle,def_60_cnt_social_circle,days_last_phone_change,flag_document_2,flag_document_3,flag_document_4,flag_document_5,flag_document_6,flag_document_7,flag_document_8,flag_document_9,flag_document_10,flag_document_11,flag_document_12,flag_document_13,flag_document_14,flag_document_15,flag_document_16,flag_document_17,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
5,228361,1,Cash loans,M,Y,N,0,153000.0,360000.0,28440.0,360000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Civil marriage,Rented apartment,0.010556,-13532,-105,-5107.0,-792,16.0,1,1,1,1,0,0,Medicine staff,2.0,3,3,THURSDAY,19,0,0,0,1,1,1,Business Entity Type 2,0.238465,0.736677,0.604113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,0.0,0.0,0.0,0.0,-386.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
11,228368,1,Cash loans,M,Y,Y,0,292500.0,1093068.0,38857.5,783000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,0.035792,-9957,-803,-4836.0,-2185,4.0,1,1,0,1,0,0,Drivers,1.0,2,2,THURSDAY,11,0,0,0,0,0,0,Self-employed,NaN,0.111394,0.198620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,3.0,1.0,3.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
13,228370,1,Cash loans,M,N,N,2,157500.0,521280.0,31630.5,450000.0,Unaccompanied,Working,Secondary / secondary special,Married,Municipal apartment,0.007114,-16533,-3963,-8792.0,-25,NaN,1,1,0,1,0,0,Laborers,4.0,2,2,TUESDAY,12,0,0,0,0,0,0,Business Entity Type 3,NaN,0.613230,0.177704,0.1608,NaN,0.9811,NaN,NaN,NaN,0.4138,0.1667,NaN,0.102,NaN,0.1523,NaN,NaN,0.1639,NaN,0.9811,NaN,NaN,NaN,0.4138,0.1667,NaN,0.1043,NaN,0.1587,NaN,NaN,0.1624,NaN,0.9811,NaN,NaN,NaN,0.4138,0.1667,NaN,0.1037,NaN,0.1551,NaN,NaN,None,block of flats,0.1198,"Stone, brick",No,0.0,0.0,0.0,0.0,-1440.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,8.0
3

In [9]:
# Get the average of defaulted loan
average_default_loan = default_loans.amt_credit.mean()
round(average_default_loan,2)

557778.53

In [10]:
# Fer the average of paid-off loan
average_normal_loan = normal_loans.amt_credit.mean()
round(average_normal_loan,2)

602648.28

Assume that on average, default happened with 90% of the credit still unpaid, and lending interest rate was 8%.

In [11]:
average_default_amount = round(average_default_loan * 0.9, 2)
average_default_amount

502000.67

In [12]:
average_lending_profit = round(average_normal_loan * 0.08, 2)
average_lending_profit

48211.86